In [32]:
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta
from Librerias_SAP import SAP_GUI
from Librerias_SAP import Funtions


In [33]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [3]:
transsaccion="ZSD035D"
variant="JESPINOSAP"
provision="INFORME FACTU"

def Search_ZSD035D(Transsaccion,session,provision,date,variant):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    if date==None:
        None
    else:
        session.findById("wnd[0]/usr/ctxtSP$00024-LOW").text = date
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    try:
        table=session.findById("wnd[0]/usr/cntlCONTAINER/shellcont/shell")
        return(table)
    except:
        return(session)

In [4]:
Search_ZSD035D(transsaccion,session,provision,"13.06.2023",variant)

<COMObject <unknown>>

In [5]:
# Pendiente Firme
Name_ZSD035D="Pendiente_Firme"  # Revisar suele no guardar
SAP_GUI.Export_TXT2(Name_ZSD035D,session)


In [6]:
# ZSD110 FIRME
url_ZSD035D="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD035D)+".txt"
Data_ZSD035D=pd.read_csv(url_ZSD035D,skiprows=5,delimiter="\t")
Data_ZSD035D=Funtions.Clean_Columns(Data_ZSD035D)
Data_ZSD035D=Data_ZSD035D[Data_ZSD035D['Material'].notnull()]
#Data_ZSD035D["Lineas"]=Data_ZSD035D["Lineas"].apply(lambda x:Funtions.Clean_num(x))


In [17]:
Data_ZSD035D.columns

Index(['Solic.', 'Solicitante', 'Pagador', 'Pagador.1', 'Destinat.',
       'Destinatario de mercancías', 'ClFac', 'OrgVt', 'CDis', 'Fecha factura',
       'Factura', 'Clase de factura', 'Tipo de lista de precios según pedido',
       'Material', 'Número de material', 'Cantidad', 'UM', 'Valor Neto',
       'Mon.', 'Pedido', 'Entrega', 'Lote', 'Nº de pedido', 'An.', 'DocFactAn',
       'Grupo art.', 'Grupo de artículos', 'CambContab', 'Código EAN/UPC',
       'Texto', 'MARCA', 'Creado el'],
      dtype='object')

In [16]:
Data_ZSD035D['Cantidad']

0        3.0
1        3.0
2        3.0
3        3.0
4        6.0
        ... 
3519    12.0
3520    12.0
3521     6.0
3522     6.0
3523    12.0
Name: Cantidad, Length: 3524, dtype: float64

In [22]:
#Limpio datos de la columnas a numeros

Data_ZSD035D['Cantidad'], Data_ZSD035D['Valor Neto']= Funtions.Clean_Num_List(Data_ZSD035D['Cantidad'],Data_ZSD035D['Valor Neto'])
Data_ZSD035D.columns


Index(['Solic.', 'Solicitante', 'Pagador', 'Pagador.1', 'Destinat.',
       'Destinatario de mercancías', 'ClFac', 'OrgVt', 'CDis', 'Fecha factura',
       'Factura', 'Clase de factura', 'Tipo de lista de precios según pedido',
       'Material', 'Número de material', 'Cantidad', 'UM', 'Valor Neto',
       'Mon.', 'Pedido', 'Entrega', 'Lote', 'Nº de pedido', 'An.', 'DocFactAn',
       'Grupo art.', 'Grupo de artículos', 'CambContab', 'Código EAN/UPC',
       'Texto', 'MARCA', 'Creado el'],
      dtype='object')

In [31]:
#Canal01_Entregado_Exluido.pivot_table(,,['Material', 'Valor Neto'])

Table_Data_ZSD035D=Data_ZSD035D.pivot_table(index=['CDis'],aggfunc={'Material':'count','Pedido':'count','Valor Neto':sum,'Cantidad':sum}).reset_index()[['CDis','Pedido', 'Material', 'Cantidad', 'Valor Neto']]
#Lineas_table_dinamica_Exito_entrega=sum(Table_Data_ZSD035D["Material"])

Table_Data_ZSD035D


,CDis,Pedido,Material,Cantidad,Valor Neto
0,1.0,1016,1016,8573.0,190189674.0
1,2.0,466,466,6401.0,76898535.0
2,7.0,907,907,7957.0,113755457.0
3,8.0,454,454,1309.0,54363944.0
4,12.0,5,5,294.0,5643408.0
5,15.0,560,560,654.0,12719902.0
6,19.0,45,45,2947.0,24367469.0
7,20.0,71,71,1164.0,6730422.0
